In [2]:
notes = """
Protracker 1.1B Song/Module Format:
-----------------------------------

Offset  Bytes  Description
------  -----  -----------
   0     20    Songname. Remember to put trailing null bytes at the end...

Information for sample 1-31:

Offset  Bytes  Description
------  -----  -----------
  20     22    Samplename for sample 1. Pad with null bytes.
  42      2    Samplelength for sample 1. Stored as number of words.
               Multiply by two to get real sample length in bytes.
  44      1    Lower four bits are the finetune value, stored as a signed
               four bit number. The upper four bits are not used, and
               should be set to zero.
               Value:  Finetune:
                 0        0
                 1       +1
                 2       +2
                 3       +3
                 4       +4
                 5       +5
                 6       +6
                 7       +7
                 8       -8
                 9       -7
                 A       -6
                 B       -5
                 C       -4
                 D       -3
                 E       -2
                 F       -1

  45      1    Volume for sample 1. Range is $00-$40, or 0-64 decimal.
  46      2    Repeat point for sample 1. Stored as number of words offset
               from start of sample. Multiply by two to get offset in bytes.
  48      2    Repeat Length for sample 1. Stored as number of words in
               loop. Multiply by two to get replen in bytes.

Information for the next 30 samples starts here. It's just like the info for
sample 1.

Offset  Bytes  Description
------  -----  -----------
  50     30    Sample 2...
  80     30    Sample 3...
   .
   .
   .
 890     30    Sample 30...
 920     30    Sample 31...

Offset  Bytes  Description
------  -----  -----------
 950      1    Songlength. Range is 1-128.
 951      1    Well... this little byte here is set to 127, so that old
               trackers will search through all patterns when loading.
               Noisetracker uses this byte for restart, but we don't.
 952    128    Song positions 0-127. Each hold a number from 0-63 that
               tells the tracker what pattern to play at that position.
1080      4    The four letters "M.K." - This is something Mahoney & Kaktus
               inserted when they increased the number of samples from
               15 to 31. If it's not there, the module/song uses 15 samples
               or the text has been removed to make the module harder to
               rip. Startrekker puts "FLT4" or "FLT8" there instead.

Offset  Bytes  Description
------  -----  -----------
1084    1024   Data for pattern 00.
   .
   .
   .
xxxx  Number of patterns stored is equal to the highest patternnumber
      in the song position table (at offset 952-1079).

Each note is stored as 4 bytes, and all four notes at each position in
the pattern are stored after each other.

00 -  chan1  chan2  chan3  chan4
01 -  chan1  chan2  chan3  chan4
02 -  chan1  chan2  chan3  chan4
etc.

Info for each note:

 _____byte 1_____   byte2_    _____byte 3_____   byte4_
/                \ /      \  /                \ /      \
0000          0000-00000000  0000          0000-00000000

Upper four    12 bits for    Lower four    Effect command.
bits of sam-  note period.   bits of sam-
ple number.                  ple number.

Periodtable for Tuning 0, Normal
  C-1 to B-1 : 856,808,762,720,678,640,604,570,538,508,480,453
  C-2 to B-2 : 428,404,381,360,339,320,302,285,269,254,240,226
  C-3 to B-3 : 214,202,190,180,170,160,151,143,135,127,120,113

To determine what note to show, scan through the table until you find
the same period as the one stored in byte 1-2. Use the index to look
up in a notenames table.

This is the data stored in a normal song. A packed song starts with the
four letters "PACK", but i don't know how the song is packed: You can
get the source code for the cruncher/decruncher from us if you need it,
but I don't understand it; I've just ripped it from another tracker...

In a module, all the samples are stored right after the patterndata.
To determine where a sample starts and stops, you use the sampleinfo
structures in the beginning of the file (from offset 20). Take a look
at the mt_init routine in the playroutine, and you'll see just how it
is done.

Refs:
https://www.ocf.berkeley.edu/~eek/index.html/tiny_examples/ptmod/ap12.html
https://forums.atariage.com/topic/157184-imagitec-designs-mod-player-informations/
"""

## Unpack a mod file

In [164]:
!ffplay ../tempest2k/src/sounds/rave4.mod

ffplay version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2003-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [4]:
tune = open("../tempest2k/src/sounds/tune1.mod","rb")

song_name = tune.read(20).replace(b'\x00',b'')

sample_name, length, fine_tune, volume, repeat_point, repeat_len = [],[],[],[],[],[]
for i in range(0,31):
    sample_name     += [tune.read(22).replace(b'\x00',b'')]
    length          += [int.from_bytes(tune.read(2), "big")]
    fine_tune       += [int.from_bytes(tune.read(1), "big")]
    volume          += [int.from_bytes(tune.read(1), "big")]
    repeat_point    += [int.from_bytes(tune.read(2), "big")]
    repeat_len      += [int.from_bytes(tune.read(2), "big")]
song_length = int.from_bytes(tune.read(1), "big")
sentinel = int.from_bytes(tune.read(1), "big")
song_positions = [x for x in tune.read(128)]
m_k = tune.read(4)

num_patterns = max(song_positions)
patterns = []
for p in range(0,num_patterns):
    chord_bytes = tune.read(1024)
    chords = []
    for n in range(0,1024,16):
        chord = []
        for o in range(n,n+16,4):
            raw_bytes = [x for x in chord_bytes[o:o+4]]
            sample_hi = raw_bytes[0] & 0xF0 >> 4
            note_period = ((raw_bytes[0] & 0x0F) << 8) + raw_bytes[1]
            sample_lo = raw_bytes[2] & 0xF0 >> 4
            effect_command = ((raw_bytes[2] & 0x0F) << 8) + raw_bytes[3]
            chord += [[sample_hi,note_period,sample_lo,effect_command]]
        chords += [chord]
    patterns += [chords]
print(patterns[2])

[[[1, 428, 15, 3852], [1, 428, 0, 0], [0, 0, 0, 0], [1, 428, 12, 3072]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 428, 0, 0], [0, 0, 10, 2576]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2576]], [[1, 428, 12, 3088], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2576]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 428, 12, 3088], [0, 0, 10, 2576]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2576]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2576]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2576]], [[1, 428, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2576]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 428, 0, 0], [0, 0, 10, 2561]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2561]], [[1, 428, 12, 3088], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2561]], [[0, 0, 0, 0], [0, 0, 0, 0], [1, 428, 12, 3088], [0, 0, 10, 2561]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2561]], [[0, 214, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 10, 2561]], [[0, 0, 0, 0], [0, 0, 0, 0]

In [66]:
"""
Periodtable for Tuning 0, Normal
  C-1 to B-1 : 856,808,762,720,678,640,604,570,538,508,480,453
  C-2 to B-2 : 428,404,381,360,339,320,302,285,269,254,240,226
  C-3 to B-3 : 214,202,190,180,170,160,151,143,135,127,120,113
"""

note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
note_table = [
    856,808,762,720,678,640,604,570,538,508,480,453,
    428,404,381,360,339,320,302,285,269,254,240,226,
    214,202,190,180,170,160,151,143,135,127,120,113]
len(note_table)

36

In [84]:
def get1Note(n,o):
    x=856<<6
    for i in range(0,n):
        x = (x*61858) >> 16
    x= x >> (o+4)
    return x

get1Note(3,1)

1439